In [1]:
print("Hello World")

Hello World


In [1]:
import torch
from transformers import pipeline

d:\Installed\ana\envs\kdd_task_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_text = 'آپ کيسے ہيں'

In [10]:
# Load model for original language to english translation
src_lang_to_eng_translator = pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-en-fr")

Device set to use cpu


In [8]:
# translate from source language to english
result = src_lang_to_eng_translator('یہ ایک پھول ہے', max_length = 512)
print(result[0]['translation_text'])

It's a flower


In [13]:
# another model for translation purposes

google_translator = pipeline("text2text-generation", model="google-t5/t5-small")

Device set to use cpu


In [21]:
result = google_translator('Translate English to Urdu: ' + 'How are you?')

In [22]:
print(result)

[{'generated_text': 'Translate English to Urdu: How are you?'}]


In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="facebook/m2m100_418M")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


In [ ]:
pipe('آپ کيسے ہيں', forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))

[{'generated_text': 'You are here'}]

In [13]:
# src lang to eng lang to src lang pipeline

test_text = 'Je m\'appelle Jhon'

original_to_en = pipe(test_text, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))
print(original_to_en[0]['generated_text'])

en_to_original = src_lang_to_eng_translator(original_to_en[0]['generated_text'], max_length = 512)
print(en_to_original[0]['translation_text'])

My name is Jhon
Mon nom est Jhon.


In [25]:
# now have to make validation pipeline
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [29]:
tokenized_reference = word_tokenize(test_text)
tokenized_candidate = word_tokenize(en_to_original[0]['translation_text'])


In [30]:
print(tokenized_reference)
print(tokenized_candidate)

['Je', "m'appelle", 'Jhon']
['Mon', 'nom', 'est', 'Jhon', '.']


In [40]:
bleu_score = sentence_bleu([tokenized_reference], tokenized_candidate)
print(bleu_score)

1.2183324802375697e-231


d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\Installed\ana\envs\kdd_task_1\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use Smoo

In [35]:
ed_sent_1_2 = nltk.edit_distance(tokenized_reference, tokenized_candidate)

In [41]:
normalized_edit_distance = (ed_sent_1_2 / max(len(tokenized_reference), len(tokenized_candidate)))
score = 1 - normalized_edit_distance

In [ ]:
print(score)

0.19999999999999996


In [42]:
explain_score = (1 - normalized_edit_distance + bleu_score) / 2
print(explain_score)

0.09999999999999998


In [43]:
print("\nMismatch Indicator:")
print("Original:        ", ' '.join(tokenized_reference))
print("Back-Translated: ", ' '.join(tokenized_candidate))

# Simple mismatch pointer
mismatch_line = []
max_len = max(len(tokenized_candidate), len(tokenized_candidate))
for i in range(max_len):
    if i >= len(tokenized_reference) or i >= len(tokenized_candidate):
        mismatch_line.append("^")
    elif tokenized_reference[i] != tokenized_candidate[i]:
        mismatch_line.append("^")
    else:
        mismatch_line.append(" ")

print("                 ", ' '.join(mismatch_line))



Mismatch Indicator:
Original:         Je m'appelle Jhon
Back-Translated:  Mon nom est Jhon .
                  ^ ^ ^ ^ ^
